In [1]:
%matplotlib inline
import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')
sys.path.append('../../../../')
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from PyFin.api import *
from alphamind.api import *
from conf.models import *
from conf.config import*
from data.engines.model import Record
from alphamind.execution.naiveexecutor import NaiveExecutor
from stacking import factor_store, feature_list
from optimization.bayes_optimization_xgb import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)

from data.engines.sqlengine import SQLEngine

weekly = 's_1'  # s1, s2,  s3, s4, s5, s_1
record_engine = SQLEngine('sqlite:///./{}/real_tune_record_without_alpha.db'.format(weekly))


In [4]:
import datetime
# ref_date_pre = '2019-12-27'

previous_pos = record_engine.fetch_record_meta(Record, trade_date=None)
previous_pos

,trade_date,weight,industry,er,code
0,2019-12-27 00:00:00.000000,0.026101,电子,0.008423,603501
1,2019-12-27 00:00:00.000000,0.025664,机械设备,0.006470,300450
2,2019-12-27 00:00:00.000000,0.024882,医药生物,0.004850,2821
3,2019-12-27 00:00:00.000000,0.024474,通信,0.001900,300383
4,2019-12-27 00:00:00.000000,0.024313,农林牧渔,0.001548,600298
5,2019-12-27 00:00:00.000000,0.024313,传媒,0.003057,300315
6,2019-12-27 00:00:00.000000,0.024294,建筑材料,0.002653,600801
7,2019-12-27 00:00:00.000000,0.024294,电子,0.002139,2273
8,2019-12-27 00:00:00.000000,0.024223,电子,0.007123,300088
9,2019-12-27 00:00:00.000000,0.024172,交通运输,0.003986,601872


In [5]:
pos_record = record_engine.fetch_record('pos_record')
# previous_pos = pos_record[pos_record['trade_date'] == ref_date_pre]
# previous_pos
pos_record

,weight,industry,er,code,trade_date
0,0.026101,电子,0.008423,603501,2019-12-27
1,0.025664,机械设备,0.006470,300450,2019-12-27
2,0.024882,医药生物,0.004850,2821,2019-12-27
3,0.024474,通信,0.001900,300383,2019-12-27
4,0.024313,农林牧渔,0.001548,600298,2019-12-27
5,0.024313,传媒,0.003057,300315,2019-12-27
6,0.024294,建筑材料,0.002653,600801,2019-12-27
7,0.024294,电子,0.002139,2273,2019-12-27
8,0.024223,电子,0.007123,300088,2019-12-27
9,0.024172,交通运输,0.003986,601872,2019-12-27
